<a href="https://colab.research.google.com/github/Christinechen66/Fraud_Detection/blob/main/Copy_of_outlier_detection_fraud_data_python3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Please ***DO NOT*** edit/run this notebook directly, otherwise others will have trouble rerunning the cells!




please click File -> save a copy in your own drive or download instead, then play with your copy! Thanks!

It is important that credit card companies are able to recognize fraudulent credit card transactions so that customers are not charged for items that they did not purchase.
where we have 492 frauds out of 284,807 transactions


In [7]:
import matplotlib.pyplot as plt
import seaborn as sns, numpy as np

from numpy import genfromtxt
from scipy.stats import multivariate_normal
from sklearn.metrics import f1_score
import io
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix


# Data Import

In [8]:
# method 1: save the data in https://drive.google.com/drive/folders/1h7dB3SLd6HYJAisALnMax28r1idDS-N6 to your own google drive: 
# create a folder: fraudData in https://drive.google.com/drive/u/0/my-drive (the home folder of gDrive is: My Drive), put the file: creditcard.csv in folder fraudData
# then follow Garima Jain in https://stackoverflow.com/questions/48340341/read-csv-to-dataframe-in-google-colab

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
!ls "/content/drive/My Drive/Colab Notebooks/creditcard.csv"

'/content/drive/My Drive/Colab Notebooks/creditcard.csv'


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
creditcardDF = pd.read_csv('drive/My Drive/fraudData/creditcard.csv') #much faster once saved in drive
creditcardDF.head()#all numerical

FileNotFoundError: ignored

In [ ]:
#method 2a, very slow, not suggested
#download the creditcard.csv to your local machine, then upload it:

from google.colab import files
uploaded = files.upload()
# It will prompt you to select a file. (may need to rerun this cell to enable this). Click on “Choose Files” then select and upload the file. Wait for the file to be 100% uploaded. 
# You should see the name of the file once Colab has uploaded it. this may take up to 15 mins

In [ ]:
##method 2b, Finally, type in the following code to import it into a dataframe (make sure the filename matches the name of the uploaded file).


creditcardDF = pd.read_csv(io.BytesIO(uploaded['creditcard.csv']))

creditcardDF.head()

In [ ]:
#Distribution of the label column
creditcardDF['Class'].value_counts()

In [ ]:
492 / (284315  + 492) #<0.2 percent

In [ ]:
creditcardDF.shape

In [ ]:
creditcardDF.isna().sum()#null checking

In [ ]:


sns.distplot(creditcardDF['Time'])

In [ ]:

sns.distplot(creditcardDF['Amount'])

In [ ]:
creditcardDF['Amount'] = np.log(creditcardDF['Amount'] + 1)

creditcardDF['Time'] = np.log(creditcardDF['Time'] + 1)

normal = creditcardDF[creditcardDF['Class'] == 0]
anomaly = creditcardDF[creditcardDF['Class'] == 1]

train, test = train_test_split(normal, test_size=0.2, random_state=0)
normal_valid, normal_test = train_test_split(test, test_size=0.5, random_state=0)#all normal/good
anomaly_valid, anomaly_test = train_test_split(anomaly, test_size=0.5, random_state=0)#all anomaly/bad

validation = pd.concat([normal_valid, anomaly_valid])#include both good and bad
test = pd.concat([normal_test, anomaly_test])#include both good and bad


print(validation.shape)
print(test.shape)

train = train.drop(columns = ['Class']).reset_index(drop= True)#no need of label in train data, drop it
print(train.shape)

In [ ]:
featureNames = list(train.columns.values)#feature names only, no label
valFeatures = validation[featureNames].reset_index(drop= True)#feature df only, no label
testFeatures = test[featureNames].reset_index(drop= True)#feature df only, no label

valLabel = validation['Class']#label df only
testLabel = test['Class']#label df only



In [ ]:
valFeatures.head()

In [ ]:
validation['Class'].value_counts()

In [ ]:
test['Class'].value_counts()




In [ ]:
plt.figure()
plt.title("train data of V1 and V2")#may contain outliers
plt.xlabel("V1")
plt.ylabel("V2")
plt.plot(train.iloc[:, 1],train.iloc[:,2],"bx")
plt.show()




In [ ]:
plt.figure()
plt.title("validation data of V1 and V2")
plt.xlabel("V1")
plt.ylabel("V2")
plt.plot(validation.iloc[:, 1],validation.iloc[:,2],"bx")
plt.show()

In [ ]:
plt.figure()
plt.title("test data of V1 and V2")
plt.xlabel("V1")
plt.ylabel("V2")
plt.plot(test.iloc[:, 1],test.iloc[:,2],"bx")
plt.show()

In [ ]:
np.arange(1, 20, 2)

In [ ]:
# def read_dataset(filePath, delimiter=','):
#     return genfromtxt(filePath, delimiter=delimiter)

#find parameter for each col/feature in df for the Gaussian distribution
def estimateGaussian(dataset):
    mu = np.mean(dataset, axis=0)
    sigma = np.cov(dataset.T)
    return mu, sigma


In [ ]:
#https://docs.scipy.org/doc/numpy/reference/generated/numpy.linspace.html
#Return evenly spaced numbers over a specified interval.
np.linspace(1, 21,10, endpoint = False)

In [ ]:
#step 1: model training
mu, sigma = estimateGaussian(train)

#generate The probability density function/curve/surface/model for multivariate_normal given mean and cov 
model = multivariate_normal(mean=mu, cov=sigma, allow_singular = True)#surface hat


In [ ]:

pdfVal = model.pdf(valFeatures)
print(max(pdfVal))#too small, can not differentiate
print(min(pdfVal))

p_val = model.logpdf(valFeatures)#Log of the pdf first, then apply to features, to change the magnitude of prob
print(max(p_val))#
print(min(p_val))

In [ ]:
p = model.logpdf(train)
print(p.shape)
print((p_val.shape))

# print(p)


In [ ]:
print(p_val)
print(p_val < -500)

In [ ]:


scores = []
p_val = model.logpdf(valFeatures)#Log of the pdf

# thresholds = np.linspace(-1000, -10,150)
thresholds = np.linspace(min(p_val), max(p_val),200)#generate all candidate threshold, epsilon

#step 2: CV to find optimal threshold: bestThreshold
for threshold in thresholds:
  y_pred = (p_val < threshold).astype(int)# list of 0 and 1
  #calculate recall, precision and f1 for each (truth, pred) pair, corresponding to that threshold
  scores.append([recall_score(valLabel, y_pred),
                 precision_score(valLabel, y_pred),
                 f1_score(valLabel, y_pred, average = "binary")])

scores = np.array(scores)
maxIndex = scores[...,2].ravel().argmax()#maxIndex of the column 2(f1_score) #193
bestThreshold = thresholds[maxIndex]
print(scores.shape)#each row is a pair of (recall, precision, f1) corresponding to a threshold

In [ ]:
print(scores)

In [ ]:
print(maxIndex)
print(bestThreshold)

In [ ]:
np.mean(train.iloc[:,1])

In [ ]:
mu[1]

In [ ]:
print(mu)
# print(sigma)




In [ ]:
#performance on test data
#step 3: prediction on test data
y_test_pred_raw = model.logpdf(testFeatures)
y_pred_test = y_test_pred_raw < bestThreshold

f1_score(testLabel, y_pred_test, average = "binary")

In [ ]:
y_pred_test

In [ ]:
#index of predicted outliers in test data
predoutliersTest = np.asarray(np.where(y_pred_test))

len(predoutliersTest[0])

In [ ]:
predoutliersTest

In [ ]:
#outliers identified on test data feature column V1 V2
plt.figure()
plt.title("test_data with outlier flaged red")
plt.xlabel("V1")
plt.ylabel("V2")
plt.plot(testFeatures.iloc[:, 1],testFeatures.iloc[:,2],"bx")#plot v1 v2 as blue cross
plt.plot(testFeatures.iloc[predoutliersTest[0],0],testFeatures.iloc[predoutliersTest[0],1],"ro")#re-plot the outliers as red dots
plt.show()

In [ ]:
#outliers identified on test data feature column V2 V3
plt.figure()
plt.title("test_data with outlier flaged red")
plt.xlabel("V2")
plt.ylabel("V3")
plt.plot(testFeatures.iloc[:, 2],testFeatures.iloc[:,3],"bx")
plt.plot(testFeatures.iloc[predoutliersTest[0],0],testFeatures.iloc[predoutliersTest[0],1],"ro")
plt.show()

In [ ]:


# generate evaluation metrics
print("%s: %r" % ("accuracy_score is: ", accuracy_score(testLabel, y_pred_test)))
print("%s: %r" % ("roc_auc_score is: ", roc_auc_score(testLabel, y_test_pred_raw)))
print("%s: %r" % ("f1_score is: ", f1_score(testLabel, y_pred_test )))#string to int

print ("confusion_matrix is: ")
cm = confusion_matrix(testLabel, y_pred_test)
cmDF = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
print(cmDF)
print('recall =',float(cm[1,1])/(cm[1,0]+cm[1,1]))
print('precision =', float(cm[1,1])/(cm[1,1] + cm[0,1]))#1.0



In [ ]:
#note we can apply supervised learning on this data too, but performance may be different, 
#can try use the output prob as an additional input feature of the supervised learning model, and see if it improved the model perf